# Hist Design Prototype

This is `fill` method in python loop:

In [ ]:
import numpy as np
import numba as nb
from hist import Hist
from hist import axis

# assets
array = np.random.randn(
    10000,
)
h = Hist.new.Reg(100, -3, 3, name="x", label="x-axis").Double()

In [ ]:
# python fill
h.fill(array)
h

In [ ]:
import hist

isinstance(h.axes[0], hist.axis.Regular)

## Numba: Hist

To extend the Numba, we first need to create a Hist type `HistType` for `Hist`, and then teach Numba about our type inference additions:

In [ ]:
from numba import types
from numba.extending import typeof_impl, as_numba_type, type_callable

# create Numba type
class HistType(types.Type):
    def __init__(self):
        super().__init__(name="Hist")


hist_type = HistType()

# infer values
@typeof_impl.register(Hist)
def typeof_index(val, c):
    return hist_type


# infer annotations
as_numba_type.register(Hist, hist_type)

# infer operations
@type_callable(Hist)
def type_hist(context):
    def typer(axes):
        for ax in axes:
            #             if not (isinstance(ax, types of axis)):
            return typer

    return hist_type

We also need to teach Numba how to actually generate native representation for the new operations:

In [ ]:
from numba.core import cgutils
from numba.extending import (
    models,
    register_model,
    make_attribute_wrapper,
    overload_attribute,
    lower_builtin,
    box,
    unbox,
    NativeValue,
)

# define data model
@register_model(HistType)
class HistModel(models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [
            #             ("axes", types of list of axis),
        ]
        models.StructModel.__init__(self, dmm, fe_type, members)


# expose attributes, porperties and constructors
make_attribute_wrapper(HistType, "axes", "axes")


@lower_builtin(Regular, types.Integer, types.Float, types.Float)
def impl_h(context, builder, sig, args):
    axes = args
    typ = sig.return_type
    h = cgutils.create_struct_proxy(typ)(context, builder)
    h.axes = axes
    return reg._getvalue()


# unbox and box
@unbox(HistType)
def unbox_h(typ, obj, c):
    axes_obj = c.pyapi.object_getattr_string(obj, "axes")
    h = cgutils.create_struct_proxy(typ)(c.context, c.builder)
    #     h.axes = c.pyapi.float_as_double(axes_obj)
    c.pyapi.decref(axes_obj)
    is_error = cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return NativeValue(h._getvalue(), is_error=is_error)


@box(HistType)
def box_h(typ, val, c):
    h = cgutils.create_struct_proxy(typ)(c.context, c.builder, value=val)
    axes_obj = c.pyapi.float_from_double(h.axes)
    class_obj = c.pyapi.unserialize(c.pyapi.serialize_object(Hist))
    res = c.pyapi.call_function_objargs(class_obj, (axes_obj))
    c.pyapi.decref(axes_obj)
    c.pyapi.decref(class_obj)
    return res

In [ ]:
reg_ax = axis.Regular(10, 0, 1)


@nb.jit
def nb_create_Hist():
    Hist(reg_ax)


nb_create_Hist()

In [ ]:
# Numba fill
# nb_fill(h, array)
# h